In [1]:
from utils import *
import numpy as np
import pickle
# import torch

In [2]:
index_to_english_alphabet=pickle.load(open('vocab_tools/index_to_english_alphabet.pickle', 'rb'))
index_to_hindi_alphabet=pickle.load(open('vocab_tools/index_to_hindi_alphabet.pickle', 'rb'))

In [3]:
hindi_alphabet_to_index=pickle.load(open('vocab_tools/hindi_alphabet_to_index.pickle', 'rb')) 

In [4]:
X_train=np.load('simple_data/X_train.npy')
X_valid=np.load('simple_data/X_val.npy')

y_train=np.load('simple_data/y_train.npy')
y_valid=np.load('simple_data/y_val.npy')

In [5]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

/opt/anaconda/envs/shared_conda_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
class Eng_Hind_Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, in_file, out_file, root_dir='simple_data',device='cuda'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.input = torch.tensor(np.load(root_dir+'/'+in_file))
        self.output = torch.tensor(np.load(root_dir+'/'+out_file))
        
        assert(len(self.input)==len(self.output),"Error: I/O Lengths must be same")
        

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X=self.input[idx]
        X=X.to(device)
        y=self.output[idx]
        y=y.to(device)
        


        sample = {'input': X, 'output': y}

        return sample

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cuda')

In [9]:
training_data=Eng_Hind_Dataset("X_train.npy","y_train.npy",device=device)
val_data=Eng_Hind_Dataset("X_val.npy","y_val.npy",device=device)
test_data=Eng_Hind_Dataset("X_test.npy","y_test.npy",device=device)

In [10]:
training_data[0]['input'].device

device(type='cuda', index=0)

In [11]:
train_dataloader = DataLoader(training_data, batch_size=16,shuffle=True)

In [12]:
val_dataloader = DataLoader(val_data, batch_size=16,shuffle=True)

In [13]:
import torch.nn as nn

In [14]:
def cell_type(mode:str='rnn'):
    mode=mode.lower()
    if mode == 'rnn':
        return nn.RNN
    elif mode =='gru':
        return nn.GRU
    else:
        return nn.LSTM
        

In [15]:
next(iter(train_dataloader))['input'].shape

torch.Size([16, 30])

In [16]:
class Encoder(nn.Module):
    """
    Input :
        - source batch
    Layer : 
        source batch -> Embedding -> LSTM
    Output :
        - LSTM hidden state
        - LSTM cell state

    Parmeters
    ---------
    input_dim : int
        Input dimension, should equal to the source vocab size.
    
    emb_dim : int
        Embedding layer's dimension.
        
    hid_dim : int
        LSTM Hidden/Cell state's dimension.
        
    n_layers : int
        Number of LSTM layers.
        
    dropout : float
        Dropout for the LSTM layer.
    """

    def __init__(self, input_size, embed_size, hid_size, num_layers, cell_mode, dropout, is_bi):
        super().__init__()
        
        # create embedding layer
        self.embedding = nn.Embedding(input_size, embed_size)

        #creating LSTM/GRU/RNN cell
        cell=cell_type(cell_mode)
        
        self.cell=cell(embed_size,hid_size,num_layers,dropout=dropout,bidirectional=is_bi)
        self.cell_mode=cell_mode
        
        

    def forward(self, input_batch: torch.LongTensor):
        """

        Parameters
        ----------
        src_batch : 2d torch.LongTensor
            Batched tokenized source sentence of shape [sent len, batch size].

        Returns
        -------
        hidden, cell : 3d torch.LongTensor
            Hidden and cell state of the LSTM layer. Each state's shape
            [n layers * n directions, batch size, hidden dim]
        """
        embedded = self.embedding(input_batch) # [sent len, batch size, emb dim]
        if self.cell_mode.lower()=='lstm':
            outputs, (hidden, cell) = self.cell(embedded)
            
        else:
            outputs, hidden = self.cell(embedded)
        # outputs -> [sent len, batch size, hidden dim * n directions]
        return hidden, cell

In [17]:
class Decoder(nn.Module):
    """
    Input :
        - first token in the target batch
        - LSTM hidden state from the encoder
        - LSTM cell state from the encoder
    Layer :
        target batch -> Embedding -- 
                                   |
        encoder hidden state ------|--> LSTM -> Linear
                                   |
        encoder cell state   -------
        
    Output :
        - prediction
        - LSTM hidden state
        - LSTM cell state

    Parmeters
    ---------
    output : int
        Output dimension, should equal to the target vocab size.
    
    emb_dim : int
        Embedding layer's dimension.
        
    hid_dim : int
        LSTM Hidden/Cell state's dimension.
        
    n_layers : int
        Number of LSTM layers.
        
    dropout : float
        Dropout for the LSTM layer.
    """

    
    def __init__(self, output_size, embed_size, hid_size, num_layers, cell_mode, dropout, is_bi):
        super().__init__()


        self.embedding = nn.Embedding(output_size, embed_size)
        
        cell=cell_type(cell_mode)
        
        self.cell=cell(embed_size,hid_size,num_layers,dropout=dropout,bidirectional=is_bi)
        self.out = nn.Linear(hid_size, output_size)
        
        self.output_size=output_size
        self.cell_mode=cell_mode

    def forward(self, trg: torch.LongTensor, hidden: torch.FloatTensor, cell: torch.FloatTensor):
        """

        Parameters
        ----------
        trg : 1d torch.LongTensor
            Batched tokenized source sentence of shape [batch size].
            
        hidden, cell : 3d torch.FloatTensor
            Hidden and cell state of the LSTM layer. Each state's shape
            [n layers * n directions, batch size, hidden dim]

        Returns
        -------
        prediction : 2d torch.LongTensor
            For each token in the batch, the predicted target vobulary.
            Shape [batch size, output dim]

        hidden, cell : 3d torch.FloatTensor
            Hidden and cell state of the LSTM layer. Each state's shape
            [n layers * n directions, batch size, hidden dim]
        """
        # [1, batch size, emb dim], the 1 serves as sent len
        embedded = self.embedding(trg.unsqueeze(0))
        if self.cell_mode.lower()=='lstm':
            outputs, (hidden, cell) = self.cell(embedded, (hidden, cell))
        else:
            outputs, hidden = self.cell(embedded, hidden)
        prediction = self.out(outputs.squeeze(0))
        return prediction, hidden, cell

In [18]:
torch.tensor([1]).to(device)

tensor([1], device='cuda:0')

In [19]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, source_batch, target_batch, teacher_forcing_ratio=0.5):

        max_len, batch_size = target_batch.shape
        target_vocab_size = self.decoder.output_size

        # tensor to store decoder's output
        outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(self.device)

        # last hidden & cell state of the encoder is used as the decoder's initial hidden state
        hidden, cell = self.encoder(source_batch)

        trg = target_batch[0]
        for i in range(1, max_len):
            prediction, hidden, cell = self.decoder(trg, hidden, cell)
            outputs[i] = prediction

            if random.random() < teacher_forcing_ratio:
                trg = target_batch[i]
            else:
                trg = prediction.argmax(1)

        return outputs


        
        

In [20]:
def train(seq2seq, iterator, optimizer, criterion):
    
    seq2seq.train()

    epoch_loss = 0
    correct=0
    for batch in iterator:
        optimizer.zero_grad()
        outputs = seq2seq(batch['input'], batch['output'])

        # 1. as mentioned in the seq2seq section, we will
        # cut off the first element when performing the evaluation
        # 2. the loss function only works on 2d inputs
        # with 1d targets we need to flatten each of them
        


        
        outputs_flatten = outputs.view(-1, outputs.shape[-1])

        
        trg_flatten = batch['output'].view(-1)
        
        loss = criterion(outputs_flatten, trg_flatten)
        
        outputs_flatten_argmax=outputs_flatten.argmax(1)
        
        
        if outputs_flatten_argmax.all() ==trg_flatten.all():
            correct+=1

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator), correct/len(iterator)

In [21]:


def evaluate(seq2seq, iterator, criterion):
    seq2seq.eval()

    epoch_loss = 0
    correct=0
    with torch.no_grad():
        for batch in iterator:
            # turn off teacher forcing
            outputs = seq2seq(batch['input'], batch['output'], teacher_forcing_ratio=0) 

            # trg = [trg sent len, batch size]
            # output = [trg sent len, batch size, output dim]
            outputs_flatten = outputs.view(-1, outputs.shape[-1])
            trg_flatten = batch['output'].view(-1)
            loss = criterion(outputs_flatten, trg_flatten)
            outputs_flatten_argmax=outputs_flatten.argmax(1)
            if outputs_flatten_argmax.all() ==trg_flatten.all():
                correct+=1
            
            epoch_loss += loss.item()

    return epoch_loss / len(iterator), correct/len(iterator)



In [23]:
def epoch_time(start_time, end_time):
    e_time = end_time - start_time
    mins = e_time // 60
    secs = e_time%60
    return mins, secs,

In [24]:
def count_params(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)

In [25]:
import time
import random

In [145]:
import torch.optim as optim
def make_model(train_iterator,valid_iterator,N_EPOCHS=20):
    E=Encoder(30,50,50,2,'lstm',0.1,False)
    E=E.to(device)
    D=Decoder(68,50,50,2,'lstm',0.1,False)
    D=D.to(device)
    S=Seq2Seq(E,D,device)
    S.to(device)    
    print(f'The model has {count_params(S):,} trainable parameters')
    
    optimizer = optim.Adam(S.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=hindi_alphabet_to_index['.'])
    criterion=criterion.to(device)
    
    best_valid_loss = float('inf')

    for epoch in range(N_EPOCHS):    
        start_time = time.time()
        train_loss,train_acc = train(S, train_iterator, optimizer, criterion)
        valid_loss,valid_acc = evaluate(S, valid_iterator, criterion)
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(S.state_dict(), 'model1.pt')

        # it's easier to see a change in perplexity between epoch as it's an exponential
        # of the loss, hence the scale of the measure is much bigger
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {valid_acc*100:.2f}%')
        
    return S

    

In [146]:
train_iterator=train_dataloader
valid_iterator=val_dataloader
SS=make_model(train_iterator,valid_iterator,N_EPOCHS=30)

The model has 89,968 trainable parameters
Epoch: 01 | Time: 0.0m 14.80s
	Train Loss: 2.936 | Train Acc: 12.56%
	 Val. Loss: 2.833 |  Val. PPL: 16.80%
Epoch: 02 | Time: 0.0m 14.79s
	Train Loss: 2.884 | Train Acc: 20.97%
	 Val. Loss: 2.800 |  Val. PPL: 38.67%
Epoch: 03 | Time: 0.0m 14.81s
	Train Loss: 2.854 | Train Acc: 33.94%
	 Val. Loss: 2.764 |  Val. PPL: 49.22%
Epoch: 04 | Time: 0.0m 14.82s
	Train Loss: 2.822 | Train Acc: 46.59%
	 Val. Loss: 2.736 |  Val. PPL: 55.08%
Epoch: 05 | Time: 0.0m 14.82s
	Train Loss: 2.792 | Train Acc: 52.28%
	 Val. Loss: 2.711 |  Val. PPL: 67.58%


KeyboardInterrupt: 

In [142]:


def evaluate_neo(seq2seq, iterator, criterion):
    seq2seq.eval()

    epoch_loss = 0
    correct=0
    wordlet=[]
    with torch.no_grad():
        for batch in iterator:
            # turn off teacher forcing
            outputs = seq2seq(batch['input'], batch['output'], teacher_forcing_ratio=0) 

            # trg = [trg sent len, batch size]
            # output = [trg sent len, batch size, output dim]
            outputs_flatten = outputs[1:].view(-1, outputs.shape[-1])
            trg_flatten = batch['output'][1:].view(-1)
            loss = criterion(outputs_flatten, trg_flatten)
            outputs_argmax=outputs.argmax(2)
            outputs_flatten_argmax=outputs_flatten.argmax(1)
            
            
            
#             print(outputs_argmax.shape)
            print(outputs_flatten_argmax.shape)
#             print(batch['output'].shape)
#             print('_')
#             print(outputs_flatten.all()==outputs_argmax.all())
#             print(outputs_flatten.all()==trg_flatten.all())
            
#             if outputs_argmax.all() ==batch['output'][1:].all():
#             print(outputs_flatten_argmax.shape)
#             print(trg_flatten.shape)
            if outputs_flatten_argmax.all() ==trg_flatten.all():
                print('-')
                print(outputs_flatten_argmax[:10])
                print(trg_flatten[:10])
                
                
            
            
#                 print(word_from_batch(batch['output']))
                print( outputs_flatten_argmax.all() ==trg_flatten.all())
#                 print(word_from_batch(outputs_argmax))
                correct+=1
                print('-')
    
            
            epoch_loss += loss.item()

    return epoch_loss / len(iterator), correct/len(iterator)



In [143]:
criteriona = nn.CrossEntropyLoss()
criteriona=criteriona.to(device)
evaluate_neo(SS,valid_iterator,criteriona)


torch.Size([450])
-
tensor([64, 26,  3,  1, 24, 65, 65, 66, 66, 66], device='cuda:0')
tensor([64, 26,  3, 18, 17, 24,  8, 65, 66, 66], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64, 26, 16,  6,  0,  6, 65, 65, 66, 66], device='cuda:0')
tensor([64, 26,  3, 16, 23,  6, 16, 65, 66, 66], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64,  7,  1,  6,  6, 12, 65, 65,  6, 65], device='cuda:0')
tensor([64, 48, 12,  3, 12, 14, 16,  6, 32, 65], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64,  4,  6,  8, 65, 65, 65, 65, 66, 66], device='cuda:0')
tensor([64,  4, 39, 29, 30,  6,  2, 21, 65, 66], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64, 16, 14, 12,  6, 26,  1, 65, 65, 66], device='cuda:0')
tensor([64, 16, 14, 12, 26,  1,  7, 17, 65, 66], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64, 26,  1,  2,  3,  3, 65, 66, 66, 66], device='cud

tensor([64, 16,  2,  1, 66, 66, 66, 66, 66, 66], device='cuda:0')
tensor([64, 16, 17,  2, 26, 65, 66, 66, 66, 66], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64,  7,  6,  3, 16,  6, 65, 65, 66, 66], device='cuda:0')
tensor([64,  7, 18, 16,  6,  2,  6, 65, 66, 66], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64,  8,  6,  8,  8, 65,  0, 65, 14, 66], device='cuda:0')
tensor([64,  8,  6, 11,  3,  8,  1, 23, 14, 12], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64, 26,  3,  6,  6,  1, 65, 66, 66, 66], device='cuda:0')
tensor([64, 26,  3, 18, 33, 18,  1,  7, 65, 66], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64, 26,  8,  6,  8,  7,  6, 65, 66, 66], device='cuda:0')
tensor([64, 22,  8,  3,  7,  8,  7, 65, 66, 66], device='cuda:0')
tensor(True, device='cuda:0')
-
torch.Size([450])
-
tensor([64, 31, 27, 15, 21, 26, 66, 66, 66, 66], device='cuda:0')
tensor([64, 31

(0.7768968029413372, 0.86328125)

In [153]:
>>> a = torch.randn(4, 4)
print(a)
a.argmax(2)

tensor([[-1.0110, -1.7012,  0.5845,  1.3827],
        [ 0.9677, -1.6683, -0.6714, -0.1745],
        [-0.7388, -0.9713,  0.2337,  1.1083],
        [-1.0661,  0.0750, -0.1140, -0.7880]])


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [27]:

# # ignore the pcriterion = nn.CrossEntropyLoss(adding index when calculating the loss
# PAD_IDX = target.vocab.stoi['<pad>']
# criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)



In [70]:
a=next(iter(train_dataloader))

In [72]:
bya=SS(a['input'],a['output'])

In [75]:
a['output'].shape

torch.Size([16, 30])

In [76]:
bya.shape

torch.Size([16, 30, 68])

tensor([64, 16,  6,  2, 32,  6, 65,  1, 65, 66, 66, 66, 66, 66, 66, 66, 66, 66,
        66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66], device='cuda:0')

In [109]:
for i in range(16):
    print(word_from_torchies(a['output'][0],index_to_hindi_alphabet))

स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम
स्वावलंबनक्षम


In [98]:
word_from_torchies(bya.argmax(2)[],index_to_hindi_alphabet)

'वानला'

In [112]:
a['output'][0]

tensor([64, 26,  3, 16,  6, 16, 32, 10,  0,  2,  7,  3, 45, 24, 65, 66, 66, 66,
        66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66, 66], device='cuda:0')

In [121]:
bya.shape

torch.Size([16, 30, 68])

In [125]:
def word_from_batch(batch):
    wordlet=[]
    for i in range(len(batch)):
        wordlet.append(word_from_torchies(batch[i],index_to_hindi_alphabet))
    return wordlet
        

In [120]:
word_from_torchies(bya.argmax(2)[0],index_to_hindi_alphabet)

ब
Invalid Word


In [115]:
for i in range(16):
    print('................')
    print(i)
    print(word_from_torchies(a['output'][i],index_to_hindi_alphabet),\
         '---',\
         word_from_torchies(bya.argmax(2)[i],index_to_hindi_alphabet)\
         )


................
0
ब
Invalid Word
स्वावलंबनक्षम --- None
................
1
वनलालपेकी --- वानला
................
2
रफा --- ['र', 'ा', 'प', 'ि', 'ा', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
................
3
कुपित --- ['र', 'ु', '्', 'ि', 'ा', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
................
4
रुद्ररुप --- रुदाा
................
5
पदाथरें --- प्दाा
................
6
ब्रहम्बाद --- उ्दाम
................
7
उत्तमराव --- उ््ाा
................
8
जोनढाले --- ब्तदा
................
9
इसमुद्दे --- इितो्
................
10
पिटोट --- ['ब', 'ि', 'र', '्', '्', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
................
11
बीडच्या --- बे्द्
................
12
बोटियां --- बो्िर
................
13
दुल्हेड़ी --- जुलबा
................
14
संबंधीच्या --- सा

In [68]:
import math

In [95]:
def word_from_torchies(torchie1,index_toalp):
    torchie=torchie1.cpu().numpy()
    return word_from_vecs(torchie,index_toalp,False)

In [69]:
word (self, source_batch,target_batch):
    max_len, batch_size = target_batch.shape
    outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(self.device)


            # last hidden & cell state of the encoder is used as the decoder's initial hidden state
    hidden, cell = self.encoder(source_batch)

    wordet=[]


    trg = torch.tensor(hindi_alphabet_to_index['<'])
    trg=trg.to(device)
    wordet.append(index_to_hindi_alphabet(trg.cpu().numpy()))
    for i in range(1, max_len):
        prediction, hidden, cell = self.decoder(trg, hidden, cell)
        outputs[i] = prediction
        trg = prediction.argmax(1)
        wordet.append(index_to_hindi_alphabet(trg.cpu().numpy()))


    return ''.join(wordet)

In [33]:
word_from_vecs(X_valid[0],index_to_english_alphabet,False)

'bajai'

In [34]:
word_from_vecs(y_valid[0],index_to_hindi_alphabet,False)

'बजाई'

In [37]:
next(iter(train_dataloader))

{'input': tensor([[26, 21,  6,  4,  6, 10,  1,  8,  3,  6,  8, 16, 27, 28, 28, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28],
         [26, 16,  6,  4,  6,  0,  6,  2,  3,  4, 18,  9, 13,  6, 27, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28],
         [26,  7, 18, 20,  0,  4, 18, 16,  4,  6,  2, 27, 28, 28, 28, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28],
         [26, 23, 11, 16, 23, 14,  9,  1,  7, 27, 28, 28, 28, 28, 28, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28],
         [26, 21,  6, 20,  6,  5,  6, 20, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28],
         [26,  9,  8,  6,  2, 16, 17, 14,  9, 14,  8, 11, 27, 28, 28, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28],
         [26, 23, 18, 14, 20,  1,  2, 10, 27, 28, 28, 28, 28, 28, 28, 28, 28, 28,
          28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28],
    

In [ ]:
s